### Import Packages

* Jupyter notebook which highlights the integration of OMERO with Cell Profiler
* Example workflow obtained from : http://cellprofiler.org/examples/#FruitFlyCells

In [1]:
# %tb
import cellprofiler.preferences as cpprefs
cpprefs.set_headless()
import cellprofiler
import cellprofiler.image as cpi
import cellprofiler.measurement as cpmeas
import cellprofiler.module as cpm
import cellprofiler.modules
import cellprofiler.modules.loadimages as LI
import cellprofiler.pipeline as cpp

import cellprofiler.setting as cps
import cellprofiler.workspace as cpw
from cellprofiler.modules.injectimage import InjectImage
import cellprofiler.modules.makeprojection as M
import cellprofiler.measurement as cpmeas
import cellprofiler.object as cpo
import cellprofiler.workspace as cpw

from omero.gateway import BlitzGateway
import numpy as np
import os

import tempfile

### Set Cell Profiler Preferences

In [2]:
cpprefs.set_headless()
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### Method for Fetching Omero Image Plane(s) 

In [3]:
def omero_image_set(image):
    
    image_set_list = cpi.ImageSetList()
    image_set = image_set_list.get_image_set(0)
    
    pixels = image.getPrimaryPixels()
    input_pixels = pixels.getPlane(0,0,0)
    return input_pixels

### OMERO Credentials

In [51]:
HOST = 'eel.openmicroscopy.org'
USERNAME = 'user-1'
PASSWORD = ''
PORT = 4064
conn = BlitzGateway(USERNAME, PASSWORD, host=HOST, port=PORT)
conn.connect()

True

### OMERO Image ID's to analyse using CP Pipeline and Manual grouping of Images

In [52]:
omero_ids_blue = [86351, 86360]
omero_ids_green = [86359, 86358]
omero_ids_red = [86357, 86354]

omero_ids = [[86351, 86359, 86357], [86360, 86358, 86354]]

### Load Cell Profiler Pipeline

In [53]:
pipeline = cpp.Pipeline()
pipeline.load("/Users/bramalingam/Downloads/ExampleFly/ExampleFly.cppipe")
for module in pipeline.modules():
    print  module.module_name

Images
Metadata
NamesAndTypes
Groups
Crop
Crop
Crop
IdentifyPrimaryObjects
IdentifySecondaryObjects
IdentifyTertiaryObjects
MeasureObjectSizeShape
MeasureObjectIntensity
MeasureTexture
MeasureObjectNeighbors
MeasureColocalization
MeasureImageIntensity
MeasureImageQuality
CalculateMath
ClassifyObjects
GrayToColor
ExportToSpreadsheet


### Run Cell Profiler Pipeline on the OMERO Images

In [57]:
import warnings
import pandas
warnings.filterwarnings('ignore')

Nuclei = pandas.DataFrame()
Cells = pandas.DataFrame()
Cytoplasm = pandas.DataFrame()
Image = pandas.DataFrame()
for img_set_num, field in enumerate(omero_ids):
    pipeline_copy = pipeline.copy()
    for image_id in field:
        image = conn.getObject("Image", image_id)
        image_name = image.getName()

        if image_name.endswith('D.TIF'):
            Image_Name = 'OrigBlue'
        if image_name.endswith('F.TIF'):
            Image_Name = 'OrigGreen'
        if image_name.endswith('R.TIF'):
            Image_Name = 'OrigRed'
            
        inject_image_module = InjectImage(Image_Name, omero_image_set(image))
        inject_image_module.set_module_num(1) 
        pipeline_copy.add_module(inject_image_module)
    
    for i in [1,2,3,4]:
        pipeline_copy.remove_module(pipeline_copy.modules()[len(field)].module_num)

    # print pipeline.modules()
    m = pipeline_copy.run();
    
    nuc_csv = pandas.read_csv(new_output_directory + '/Nuclei.csv')
    nuc_csv.ImageNumber += img_set_num
    Nuclei = Nuclei.append(nuc_csv)
    
    cel_csv = pandas.read_csv(new_output_directory + '/Cells.csv')
    cel_csv.ImageNumber += img_set_num
    Cells = Cells.append(cel_csv)
    
    cyto_csv = pandas.read_csv(new_output_directory + '/Cytoplasm.csv')
    cyto_csv.ImageNumber += img_set_num
    Cytoplasm = Cytoplasm.append(cyto_csv)
    
    image_csv = pandas.read_csv(new_output_directory + '/Image.csv')
    image_csv.ImageNumber += img_set_num
    Image = Image.append(image_csv)

for module in pipeline_copy.modules():
    print  module.module_name

InjectImage
InjectImage
InjectImage
Crop
Crop
Crop
IdentifyPrimaryObjects
IdentifySecondaryObjects
IdentifyTertiaryObjects
MeasureObjectSizeShape
MeasureObjectIntensity
MeasureTexture
MeasureObjectNeighbors
MeasureColocalization
MeasureImageIntensity
MeasureImageQuality
CalculateMath
ClassifyObjects
GrayToColor
ExportToSpreadsheet


## Results obtained as CSV from Cell Profiler

### Measurements (Nuclei)

In [59]:
import pandas
Nuclei.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Center_Z,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,AreaShape_Extent,...,Texture_SumEntropy_CropBlue_3_02,Texture_SumEntropy_CropBlue_3_03,Texture_SumVariance_CropBlue_3_00,Texture_SumVariance_CropBlue_3_01,Texture_SumVariance_CropBlue_3_02,Texture_SumVariance_CropBlue_3_03,Texture_Variance_CropBlue_3_00,Texture_Variance_CropBlue_3_01,Texture_Variance_CropBlue_3_02,Texture_Variance_CropBlue_3_03
0,1,1,360,133.0,28.0,1.0,1.112285,0.756076,1.0,0.784314,...,6.354523,6.308452,724.563924,477.410947,598.249896,612.675279,211.659589,215.454660,210.159449,210.577060
1,1,2,325,87.0,48.0,1.0,1.088447,0.715051,1.0,0.706522,...,6.466266,6.473102,789.370104,470.203640,805.744251,844.428515,292.672293,301.859734,294.513421,291.769730
2,1,3,188,26.0,45.0,1.0,1.040556,0.645978,1.0,0.734375,...,5.532754,5.534234,218.652857,142.866497,247.581829,226.730610,94.784643,98.731975,94.219401,86.947920
3,1,4,320,14.0,66.0,1.0,1.007676,0.361018,1.0,0.800000,...,6.097452,6.000090,551.235507,394.865385,572.957751,449.026293,210.948724,221.452457,220.825976,207.832331
4,1,5,201,41.0,71.0,1.0,1.082705,0.718680,1.0,0.705263,...,5.661191,5.601019,285.654016,231.587456,310.802956,250.455748,116.600148,122.262864,108.214980,117.167641


### Measurements (Cells)

In [60]:
Cells.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Center_Z,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,AreaShape_Extent,...,Texture_SumEntropy_CropBlue_3_02,Texture_SumEntropy_CropBlue_3_03,Texture_SumVariance_CropBlue_3_00,Texture_SumVariance_CropBlue_3_01,Texture_SumVariance_CropBlue_3_02,Texture_SumVariance_CropBlue_3_03,Texture_Variance_CropBlue_3_00,Texture_Variance_CropBlue_3_01,Texture_Variance_CropBlue_3_02,Texture_Variance_CropBlue_3_03
0,1,1,360,133.0,28.0,1.0,1.112285,0.756076,1.0,0.784314,...,6.354523,6.308452,724.563924,477.410947,598.249896,612.675279,211.659589,215.454660,210.159449,210.577060
1,1,2,325,87.0,48.0,1.0,1.088447,0.715051,1.0,0.706522,...,6.466266,6.473102,789.370104,470.203640,805.744251,844.428515,292.672293,301.859734,294.513421,291.769730
2,1,3,288,25.0,46.0,1.0,1.684313,0.899192,1.0,0.342857,...,6.106281,6.047411,626.883997,642.342271,661.257337,511.905062,207.197922,216.521903,206.960273,194.885240
3,1,4,618,13.0,64.0,1.0,1.720838,0.855008,1.0,0.433380,...,6.459358,6.447722,1724.631513,1454.210508,1745.333264,1630.942752,496.518202,476.223165,497.502066,509.431746
4,1,5,366,42.0,72.0,1.0,1.416170,0.764232,1.0,0.448529,...,5.999302,5.797992,804.754601,659.061728,903.324238,910.354852,245.024227,233.200988,247.049571,264.363203


### Measurements (Cytoplasm)

In [61]:
Cytoplasm.head()

,ImageNumber,ObjectNumber,AreaShape_Area,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Center_Z,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,AreaShape_Extent,...,Texture_SumEntropy_CropBlue_3_02,Texture_SumEntropy_CropBlue_3_03,Texture_SumVariance_CropBlue_3_00,Texture_SumVariance_CropBlue_3_01,Texture_SumVariance_CropBlue_3_02,Texture_SumVariance_CropBlue_3_03,Texture_Variance_CropBlue_3_00,Texture_Variance_CropBlue_3_01,Texture_Variance_CropBlue_3_02,Texture_Variance_CropBlue_3_03
0,1,1,83,147.0,32.0,1.0,8.340673,0.731592,0.0,0.180828,...,3.000000,2.873141,10.748582,52.484375,42.437500,48.390533,6.089319,29.089844,17.859375,24.828402
1,1,2,82,95.0,48.0,1.0,7.327538,0.679348,0.0,0.178261,...,2.777363,2.913977,61.454545,59.666667,21.869822,26.793388,22.522727,36.916667,12.852071,13.993802
2,1,3,159,16.0,50.0,1.0,4.691123,0.918840,0.0,0.189286,...,4.726059,4.739991,150.801649,172.349479,175.608612,183.951325,62.882704,61.911838,61.951173,80.069227
3,1,4,374,0.0,68.0,1.0,3.928261,0.887578,0.0,0.262272,...,4.609439,4.347905,126.968046,126.727160,107.686234,105.497830,43.542982,56.883713,37.859982,39.862553
4,1,5,227,32.0,63.0,1.0,2.850767,0.733074,0.0,0.278186,...,4.370579,3.769307,134.019694,104.164895,142.787111,118.054671,47.027396,46.299698,40.883715,37.256386


In [64]:
Image.head()

,Classify_Large_NumObjectsPerBin,Classify_Large_PctObjectsPerBin,Classify_Medium_NumObjectsPerBin,Classify_Medium_PctObjectsPerBin,Classify_Small_NumObjectsPerBin,Classify_Small_PctObjectsPerBin,Correlation_Correlation_CropBlue_CropGreen,Correlation_Costes_CropBlue_CropGreen,Correlation_Costes_CropGreen_CropBlue,Correlation_K_CropBlue_CropGreen,...,Texture_Variance_CropBlue_3_02,Texture_Variance_CropBlue_3_03,Threshold_FinalThreshold_Cells,Threshold_FinalThreshold_Nuclei,Threshold_OrigThreshold_Cells,Threshold_OrigThreshold_Nuclei,Threshold_SumOfEntropies_Cells,Threshold_SumOfEntropies_Nuclei,Threshold_WeightedVariance_Cells,Threshold_WeightedVariance_Nuclei
0,3,8.823529,3,8.823529,8,23.529412,0.526074,0.990753,0.954735,0.943040,...,369.211218,372.200473,0.113534,0.058824,0.113534,0.058824,-13.206373,-12.841998,0.636118,1.342988
0,2,7.142857,1,3.571429,6,21.428571,0.178739,0.621968,0.428216,0.446542,...,550.550634,549.096159,0.086443,0.078431,0.086443,0.078431,-12.715738,-12.636375,0.401940,0.998734


### Push Results back to OMERO

In [62]:
import glob
image = conn.getObject("image", image_id)
dataset = image.getParent()

for file_to_upload in glob.glob(new_output_directory + '/*.csv'):
    
    if file_to_upload.endswith('Nuclei.csv'):
        Nuclei.to_csv(file_to_upload)
    elif file_to_upload.endswith('Cells.csv'):
        Cells.to_csv(file_to_upload)
    elif file_to_upload.endswith('Cytoplasm.csv'):
        Cytoplasm.to_csv(file_to_upload)
    elif file_to_upload.endswith('Image.csv'):
        Image.to_csv(file_to_upload)
    
    # create the original file and file annotation (uploads the file etc.)
    namespace = "cambridge.training.meeting.demo"
    print "\nCreating an OriginalFile and FileAnnotation"
    file_ann = conn.createFileAnnfromLocalFile(file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
    print "Attaching FileAnnotation to Plate: ", "File ID:", file_ann.getId(), \
        ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize()
    dataset.linkAnnotation(file_ann)


Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Plate:  File ID: 32601 , Cells.csv Size: 114748

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Plate:  File ID: 32602 , Cytoplasm.csv Size: 113373

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Plate:  File ID: 32603 , Image.csv Size: 38831

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Plate:  File ID: 32604 , Nuclei.csv Size: 127605


### Raw Script For Modules (Cell Profiler the difficult way!)

In [100]:
import numpy as np

from cellprofiler.preferences import set_headless

set_headless()

import cellprofiler.workspace as cpw
import cellprofiler.pipeline as cpp
import cellprofiler.image as cpi
import cellprofiler.modules.crop as cpmc
import cellprofiler.measurement as cpm
import cellprofiler.object as cpo
import cellprofiler.preferences as cpprefs

INPUT_IMAGE = "input_image"
CROP_IMAGE = "crop_image"
CROP_OBJECTS = "crop_objects"
CROPPING = "cropping"
OUTPUT_IMAGE = "output_image"

def crop_omero_image(image):
    
    image_set_list = cpi.ImageSetList()
    image_set = image_set_list.get_image_set(0)
    
    pixels = image.getPrimaryPixels()
    IMAGE_NAME = image.getName()
    
    output_image_dict = {}
    pixels_id = pixels.getId()
    sizeZ = pixels.getSizeZ()
    sizeC = pixels.getSizeC()
    sizeT = pixels.getSizeT()
    for c in range(0,sizeC):
        for z in range(0, sizeZ):
            for t in range(0, sizeT):
                input_pixels = pixels.getPlane(z,c,t)
                
                INPUT_IMAGE = IMAGE_NAME + '_Z_' + str(z) + '_C_' + str(c) + '_T_' + str(t)

                image_set.add(INPUT_IMAGE, cpi.Image(input_pixels))
                
                crop_image = np.zeros((image.getSizeY(), image.getSizeX()), bool)
                crop_image[1:200,1:200] = 1
                image_set.add(CROP_IMAGE, cpi.Image(crop_image))
                
                module = cpmc.Crop()
                module.module_num = 1
                module.image_name.value = INPUT_IMAGE
                module.cropped_image_name.value = OUTPUT_IMAGE
                module.image_mask_source.value = CROP_IMAGE
                
                pipeline = cpp.Pipeline()
                pipeline.add_module(module)

                object_set = cpo.ObjectSet()
                m = cpm.Measurements()

                workspace = cpw.Workspace(pipeline,
                                          module,
                                          image_set,
                                          object_set,
                                          m,
                                          image_set_list)

                m.add_measurement(cpm.IMAGE, cpm.GROUP_INDEX, 0, image_set_number=1)
                m.add_measurement(cpm.IMAGE, cpm.GROUP_NUMBER, 1, image_set_number=1)

                module.shape.value = cpmc.SH_IMAGE
                module.remove_rows_and_columns.value = cpmc.RM_NO
                module.run(workspace)
                output_image = workspace.image_set.get_image(OUTPUT_IMAGE)
                
                output_image_dict[INPUT_IMAGE] = output_image
    
    return output_image_dict

new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

output_image_dict = crop_omero_image(image)

%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
plt.subplot(1,2,1)
output_image = output_image_dict[output_image_dict.keys()[0]]
plt.imshow(output_image.get_image(), cmap='gray')
plt.axis('off')